In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = 'dataset/'
if not os.path.exists(dataset_path):
    print("Loading CIFAR-10 dataset...")
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    
    train_mask = y_train.flatten() < 4
    test_mask = y_test.flatten() < 4
    x_train = x_train[train_mask][:5000] 
    y_train = y_train[train_mask][:5000]
    x_test = x_test[test_mask]
    y_test = y_test[test_mask]
    
    y_train = tf.keras.utils.to_categorical(y_train, 4)
    y_test = tf.keras.utils.to_categorical(y_test, 4)
    
    def preprocess(x, y):
        x = tf.image.resize(x, (224, 224))
        x = x / 255.0
        return x, y
    
    train_generator = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32).map(preprocess)
    val_generator = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).map(preprocess)
    num_classes = 4
else:
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )
    train_generator = train_datagen.flow_from_directory(
        'dataset/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    val_generator = train_datagen.flow_from_directory(
        'dataset/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    num_classes = train_generator.num_classes
base_model = ResNet50(weights='imagenet',
include_top=False,
input_shape=(224,224,3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'])

print("Training with frozen base layers...")
model.fit(train_generator,
validation_data=val_generator,
epochs=5)
for layer in base_model.layers[-30:]:
 layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
loss='categorical_crossentropy',
metrics=['accuracy'])

print("Training with fine-tuning (last 30 layers unfrozen)...")
model.fit(train_generator,
validation_data=val_generator,
epochs=5)

Loading CIFAR-10 dataset...
Training with frozen base layers...
Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 515s 3s/step - accuracy: 0.2588 - loss: 1.4228 - val_accuracy: 0.2825 - val_loss: 1.3848
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 496s 3s/step - accuracy: 0.2532 - loss: 1.3850 - val_accuracy: 0.2512 - val_loss: 1.3838
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 446s 3s/step - accuracy: 0.2522 - loss: 1.3853 - val_accuracy: 0.2500 - val_loss: 1.3863
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 418s 3s/step - accuracy: 0.2518 - loss: 1.3865 - val_accuracy: 0.2500 - val_loss: 1.3863
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 417s 3s/step - accuracy: 0.2436 - loss: 1.3864 - val_accuracy: 0.2500 - val_loss: 1.3863
Training with fine-tuning (last 30 layers unfrozen)...
Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 534s 3s/step - accuracy: 0.5290 - loss: 1.0905 - val_accuracy: 0.2500 - val_loss: 1.4526
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 572s 4s/step - accuracy: 0.6350 - loss: 0.8780 - val_accuracy: 0.4290 - va